In [14]:
# Cell 1: Setup and Imports

# --- Package Installation ---
# Install necessary packages with specific versions for compatibility
%pip install tensorflow==2.15.0 keras==2.15.0 protobuf==4.25.3
%pip install yolov5 pandas opencv-python torch torchvision tqdm matplotlib Pillow

# --- Library Imports ---
import os
import shutil
import cv2
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tqdm.notebook import tqdm
from IPython.display import display, HTML

# --- Environment Configuration ---
# Suppress verbose logs for a cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings('ignore')

print("Setup complete. All libraries are installed and imported.")


Note: you may need to restart the kernel to use updated packages.
Setup complete. All libraries are installed and imported.


In [15]:
# Cell 2: Model Loading

# --- Configuration ---
YOLO_MODEL_PATH = 'best_yolo_completeDataset.pt'
CLASSIFIER_MODEL_PATH = 'ship_classifier.h5'
CLASSIFIER_LABELS_PATH = 'labels.txt'
YOLO_REPO_PATH = 'yolov5' # Assumes yolov5 repo is cloned

# --- Load YOLOv5 Detection Model ---
print(f"Loading YOLOv5 model from '{YOLO_MODEL_PATH}'...")
try:
    yolo_model = torch.hub.load(YOLO_REPO_PATH, 'custom', path=YOLO_MODEL_PATH, source='local', force_reload=True)
    if torch.cuda.is_available():
        yolo_model.to('cuda')
    print("✅ YOLOv5 model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading YOLOv5 model: {e}")

# --- Load Keras/TF Classification Model ---
print(f"Loading classifier model from '{CLASSIFIER_MODEL_PATH}'...")
try:
    classifier_model = tf.keras.models.load_model(CLASSIFIER_MODEL_PATH, compile=False)
    with open(CLASSIFIER_LABELS_PATH, 'r') as f:
        classifier_labels = [line.strip().split(' ', 1)[1] for line in f]
    print(f"✅ Classifier model loaded successfully. Classes: {classifier_labels}")
except Exception as e:
    print(f"❌ Error loading classifier model: {e}")

Loading YOLOv5 model from 'best_yolo_completeDataset.pt'...


YOLOv5  v7.0-422-g2540fd4c Python-3.10.18 torch-2.7.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


✅ YOLOv5 model loaded successfully.
Loading classifier model from 'ship_classifier.h5'...
✅ Classifier model loaded successfully. Classes: ['Ship', 'No ship']


In [16]:
# Cell 3: Helper Functions

def calculate_iou(boxA, boxB):
    """Calculates the Intersection over Union (IoU) between two bounding boxes."""
    interArea = max(0, min(boxA[2], boxB[2]) - max(boxA[0], boxA[0])) * max(0, min(boxA[3], boxB[3]) - max(boxA[1], boxA[1]))
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = float(boxAArea + boxBArea - interArea)
    return interArea / unionArea if unionArea > 0 else 0

def load_yolo_labels(label_path, img_width, img_height):
    """Loads ground truth boxes from a YOLO .txt file and denormalizes them."""
    boxes = []
    if not os.path.exists(label_path): return boxes
    with open(label_path, 'r') as f:
        for line in f:
            _, x_center, y_center, width, height = map(float, line.strip().split())
            w_abs, h_abs = width * img_width, height * img_height
            xmin = (x_center * img_width) - (w_abs / 2)
            ymin = (y_center * img_height) - (h_abs / 2)
            boxes.append([xmin, ymin, xmin + w_abs, ymin + h_abs])
    return boxes

def evaluate_performance(predictions, ground_truth, iou_threshold=0.1):
    """
    Evaluates detections against ground truth to calculate TP, FP, FN.
    Returns a dictionary of counts and the list of matches.
    """
    tp, fp = 0, 0
    gt_matched = [False] * len(ground_truth)
    matches = [] # List to store match status for each prediction

    for pred_box in predictions:
        best_iou, best_gt_idx = 0, -1
        for i, gt_box in enumerate(ground_truth):
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou, best_gt_idx = iou, i
        
        if best_iou > iou_threshold and best_gt_idx != -1 and not gt_matched[best_gt_idx]:
            tp += 1
            gt_matched[best_gt_idx] = True
            matches.append({'box': pred_box, 'status': 'TP'})
        else:
            fp += 1
            matches.append({'box': pred_box, 'status': 'FP'})
    
    fn = len(ground_truth) - sum(gt_matched)
    
    # Identify the specific FN boxes
    unmatched_gt = [gt_boxes for i, gt_boxes in enumerate(ground_truth) if not gt_matched[i]]
    for box in unmatched_gt:
        matches.append({'box': box, 'status': 'FN'})

    return {'tp': tp, 'fp': fp, 'fn': fn}, matches

def draw_results_on_image(image, matches):
    """Draws colored bounding boxes on an image based on match status."""
    color_map = {
        'TP': (0, 255, 0),   # Green
        'FP': (0, 0, 255),   # Red
        'FN': (255, 0, 0)    # Blue
    }
    for match in matches:
        xmin, ymin, xmax, ymax = map(int, match['box'])
        status = match['status']
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color_map[status], 2)
    return image

def classify_crop(crop_np, model, labels):
    """Classifies an image crop using the Teachable Machine model."""
    image = Image.fromarray(crop_np).resize((224, 224))
    image_array = np.asarray(image)
    normalized_image = (image_array.astype(np.float32) / 127.5) - 1
    data = np.expand_dims(normalized_image, axis=0)
    
    prediction = model.predict(data, verbose=0)
    index = np.argmax(prediction)
    return labels[index].lower(), prediction[0][index]

print("Helper functions defined.")

Helper functions defined.


# Cell 4: Analysis on `land_dataset`

In [18]:
# Cell to Find the Optimal Classifier Threshold

import matplotlib.pyplot as plt
import io

def find_optimal_threshold(dataset_name='land_dataset', num_images=100):
    """
    Analyzes the two-stage model's performance across various classifier
    confidence thresholds and compares it to the baseline YOLOv5 model.
    """
    display(HTML(f"<h3>Analyzing Optimal Threshold for '{dataset_name}'</h3>"))
    
    # --- STAGE 1: Pre-computation of all detections and their scores ---
    # This is done once to make the subsequent analysis much faster.
    images_path = os.path.join(f'./{dataset_name}/', 'images')
    labels_path = os.path.join(f'./{dataset_name}/', 'labels')
    image_files = sorted(os.listdir(images_path))[:num_images]

    all_detections = []
    total_gt_boxes = 0
    yolo_only_totals = {'tp': 0, 'fp': 0}

    for image_name in tqdm(image_files, desc="Stage 1: Pre-computing Detections"):
        # Load data
        image_path = os.path.join(images_path, image_name)
        label_path = os.path.join(labels_path, os.path.splitext(image_name)[0] + '.txt')
        
        original_image_rgb = np.array(Image.open(image_path).convert("RGB"))
        h, w, _ = original_image_rgb.shape
        gt_boxes = load_yolo_labels(label_path, w, h)
        total_gt_boxes += len(gt_boxes)

        # Get YOLOv5 predictions
        yolo_results = yolo_model(original_image_rgb)
        yolo_predictions = [pred[:4].tolist() for pred in yolo_results.xyxy[0].cpu().numpy()]
        
        # Evaluate YOLO-only performance for the baseline
        gt_matched_yolo = [False] * len(gt_boxes)
        for pred_box in yolo_predictions:
            is_tp = False
            for i, gt_box in enumerate(gt_boxes):
                if not gt_matched_yolo[i] and calculate_iou(pred_box, gt_box) > 0.4:
                    gt_matched_yolo[i] = True
                    is_tp = True
                    break
            if is_tp:
                yolo_only_totals['tp'] += 1
            else:
                yolo_only_totals['fp'] += 1

            # Classify the crop for the two-stage analysis
            xmin, ymin, xmax, ymax = map(int, pred_box)
            crop = original_image_rgb[ymin:ymax, xmin:xmax]
            if crop.size == 0: continue
            
            pred_class, confidence = classify_crop(crop, classifier_model, classifier_labels)
            
            all_detections.append({
                "class": pred_class,
                "confidence": confidence,
                "is_potential_tp": is_tp 
            })
    
    yolo_only_totals['fn'] = total_gt_boxes - yolo_only_totals['tp']
    print(f"Pre-computation complete. Analyzed {len(all_detections)} YOLO detections from {num_images} images.")

    # --- STAGE 2: Evaluate performance across different thresholds ---
    thresholds_to_test = np.arange(0.50, 1.0, 0.05) # Test from 0.50 to 0.95
    analysis_results = []

    for threshold in thresholds_to_test:
        tp, fp, tn = 0, 0, 0
        
        for det in all_detections:
            is_accepted = 'ship' in det['class'] and det['confidence'] >= threshold
            
            if is_accepted:
                if det['is_potential_tp']:
                    tp += 1
                else:
                    fp += 1
            else: # If discarded by the classifier
                if not det['is_potential_tp']: # It was a YOLO FP and we correctly discarded it
                    tn += 1
        
        fn = total_gt_boxes - tp
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        analysis_results.append({
            "Threshold": f"{threshold:.2f}",
            "TP": tp, "FP": fp, "FN": fn, "TN (Correct Rejections)": tn,
            "Precision": precision, "Recall": recall, "F1-Score": f1_score
        })

    # --- STAGE 3: Display Results ---
    df_results = pd.DataFrame(analysis_results)
    
    # Find the best F1-score to highlight it
    best_f1_score = df_results['F1-Score'].max()
    
    styled_df = df_results.style.apply(lambda s: ['background-color: yellow' if v == best_f1_score else '' for v in s], subset=['F1-Score'])\
                                .format({ "Precision": "{:.2%}", "Recall": "{:.2%}", "F1-Score": "{:.3f}" })

    display(HTML("<h4>Performance of Two-Stage Model vs. Classifier Threshold</h4>"))
    display(styled_df)

    # --- Baseline YOLOv5 Performance ---
    yolo_precision = yolo_only_totals['tp'] / (yolo_only_totals['tp'] + yolo_only_totals['fp'])
    yolo_recall = yolo_only_totals['tp'] / (yolo_only_totals['tp'] + yolo_only_totals['fn'])
    yolo_f1 = 2 * (yolo_precision * yolo_recall) / (yolo_precision + yolo_recall)
    
    display(HTML(f"""
    <div style="border: 1px solid #ccc; padding: 10px; margin-top: 20px;">
        <h4>Baseline Performance (YOLOv5 Only)</h4>
        <p>
            <b>Precision:</b> {yolo_precision:.2%} | 
            <b>Recall:</b> {yolo_recall:.2%} | 
            <b>F1-Score:</b> {yolo_f1:.3f}
        </p>
        <p>(TP: {yolo_only_totals['tp']}, FP: {yolo_only_totals['fp']}, FN: {yolo_only_totals['fn']})</p>
    </div>
    """))

    # --- Create and display the graph ---
    fig, ax = plt.subplots(figsize=(12, 7))
    ax.plot(df_results["Threshold"], df_results["Precision"], 'b-o', label='Precision')
    ax.plot(df_results["Threshold"], df_results["Recall"], 'g-o', label='Recall')
    ax.plot(df_results["Threshold"], df_results["F1-Score"], 'r-s', label='F1-Score', linewidth=3)
    
    # Add a line for the baseline F1-score for comparison
    ax.axhline(y=yolo_f1, color='purple', linestyle=':', label=f'YOLOv5 Only F1-Score ({yolo_f1:.3f})')
    
    best_f1_row = df_results.loc[df_results['F1-Score'].idxmax()]
    ax.axvline(x=best_f1_row['Threshold'], color='grey', linestyle='--', label=f'Best Two-Stage F1 ({best_f1_row["F1-Score"]:.3f}) at Threshold {best_f1_row["Threshold"]}')
    
    ax.set_title(f'Model Performance on "{dataset_name}"')
    ax.set_xlabel('Classifier Confidence Threshold')
    ax.set_ylabel('Score')
    ax.grid(True)
    ax.legend()
    
    # Save the plot to a memory buffer and display as an image widget
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    graph_widget = widgets.Image(value=buf.read(), format='png')
    display(graph_widget)
    buf.close()
    plt.close(fig)

# --- Run the analysis function ---
find_optimal_threshold()

Stage 1: Pre-computing Detections:   0%|          | 0/100 [00:00<?, ?it/s]

Pre-computation complete. Analyzed 465 YOLO detections from 100 images.


,Threshold,TP,FP,FN,TN (Correct Rejections),Precision,Recall,F1-Score
0,0.50,350,115,118,0,75.27%,74.79%,0.750
1,0.55,346,109,122,6,76.04%,73.93%,0.750
2,0.60,340,109,128,6,75.72%,72.65%,0.742
3,0.65,331,107,137,8,75.57%,70.73%,0.731
4,0.70,323,106,145,9,75.29%,69.02%,0.720
5,0.75,315,97,153,18,76.46%,67.31%,0.716
6,0.80,305,94,163,21,76.44%,65.17%,0.704
7,0.85,298,83,170,32,78.22%,63.68%,0.702
8,0.90,278,74,190,41,78.98%,59.40%,0.678
9,0.95,264,61,204,54,81.23%,56.41%,0.666


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xf6\x00\x00\x02r\x08\x06\x00\x00\x00nN[.\x00\x00…

In [17]:
def run_full_analysis(dataset_name, yolo_model, classifier_model, classifier_labels):
    """
    Runs the complete analysis for a given dataset and returns a results summary.
    """
    # --- Setup paths and directories ---
    DATASET_PATH = f'./{dataset_name}/'
    IMAGES_PATH = os.path.join(DATASET_PATH, 'images')
    LABELS_PATH = os.path.join(DATASET_PATH, 'labels')
    
    RESULTS_BASE_PATH = './results/'
    YOLO_ONLY_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, f'{dataset_name}_yolo_only')
    TWO_STAGE_RESULTS_PATH = os.path.join(RESULTS_BASE_PATH, f'{dataset_name}_two_stage')

    for path in [YOLO_ONLY_RESULTS_PATH, TWO_STAGE_RESULTS_PATH]:
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)

    image_files = sorted(os.listdir(IMAGES_PATH))
    
    # --- Initialize counters ---
    yolo_only_totals = {'tp': 0, 'fp': 0, 'fn': 0}
    two_stage_totals = {'tp': 0, 'fp': 0, 'fn': 0}

    # --- Main processing loop ---
    for image_name in tqdm(image_files, desc=f"Analyzing {dataset_name}"):
        image_path = os.path.join(IMAGES_PATH, image_name)
        label_path = os.path.join(LABELS_PATH, os.path.splitext(image_name)[0] + '.txt')
        
        original_image_bgr = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(original_image_bgr, cv2.COLOR_BGR2RGB)
        h, w, _ = original_image_bgr.shape
        
        gt_boxes = load_yolo_labels(label_path, w, h)
        yolo_results = yolo_model(original_image_rgb)
        yolo_predictions = [pred[:4].tolist() for pred in yolo_results.xyxy[0].cpu().numpy()]

        # 1. YOLO Only Analysis
        yolo_perf, yolo_matches = evaluate_performance(yolo_predictions, gt_boxes)
        for key in yolo_only_totals: yolo_only_totals[key] += yolo_perf[key]
        
        yolo_img_out = draw_results_on_image(original_image_bgr.copy(), yolo_matches)
        cv2.imwrite(os.path.join(YOLO_ONLY_RESULTS_PATH, image_name), yolo_img_out)

        # 2. Two-Stage Analysis (YOLO + Classifier)
        filtered_predictions = []
        for pred_box in yolo_predictions:
            xmin, ymin, xmax, ymax = map(int, pred_box)
            crop = original_image_rgb[ymin:ymax, xmin:xmax]
            if crop.size == 0: continue
            
            pred_class, confidence = classify_crop(crop, classifier_model, classifier_labels)
            # Assuming the positive class is the first one in labels.txt and contains 'ship'
            if 'ship' in pred_class and confidence > 0.95:
                filtered_predictions.append(pred_box)

        two_stage_perf, two_stage_matches = evaluate_performance(filtered_predictions, gt_boxes)
        for key in two_stage_totals: two_stage_totals[key] += two_stage_perf[key]
        
        two_stage_img_out = draw_results_on_image(original_image_bgr.copy(), two_stage_matches)
        cv2.imwrite(os.path.join(TWO_STAGE_RESULTS_PATH, image_name), two_stage_img_out)

    # --- Generate summary table ---
    def calculate_metrics(results):
        tp, fp, fn = results['tp'], results['fp'], results['fn']
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        return [tp, fp, fn, f"{precision:.2%}", f"{recall:.2%}", f"{f1_score:.3f}"]

    yolo_metrics = calculate_metrics(yolo_only_totals)
    two_stage_metrics = calculate_metrics(two_stage_totals)
    
    df = pd.DataFrame({
        'YOLOv5 Only': yolo_metrics,
        'YOLOv5 + Classifier': two_stage_metrics
    }, index=['True Positives (TP)', 'False Positives (FP)', 'False Negatives (FN)', 'Precision', 'Recall', 'F1-Score'])
    
    display(HTML(f"<h3>Performance on <u>{dataset_name}</u></h3>"))
    display(df)

# --- Run the analysis for the land dataset ---
run_full_analysis('land_dataset', yolo_model, classifier_model, classifier_labels)

Analyzing land_dataset:   0%|          | 0/562 [00:00<?, ?it/s]

,YOLOv5 Only,YOLOv5 + Classifier
True Positives (TP),1465,1275
False Positives (FP),2404,1417
False Negatives (FN),2144,2334
Precision,37.87%,47.36%
Recall,40.59%,35.33%
F1-Score,0.392,0.405


In [19]:
# Cell for Classifier Debugging (CORRECTED)

# --- Configuration for the debug cell ---
from IPython.display import display, HTML
import ipywidgets as widgets
from tqdm.notebook import tqdm

NUM_IMAGES_TO_DEBUG = 10
DATASET_TO_DEBUG = 'land_dataset'  # You can change this to 'sea_dataset'
DEBUG_CONFIDENCE_THRESHOLD = 0.50
POSITIVE_CLASS_NAME = 'ship'

# --- Debugging Logic ---
# This code assumes the models and helper functions from previous cells are already loaded in memory.
images_path = os.path.join(f'./{DATASET_TO_DEBUG}/', 'images')
image_files = sorted(os.listdir(images_path))[:NUM_IMAGES_TO_DEBUG]

# Check if models are loaded to avoid errors
if 'yolo_model' not in locals() or 'classifier_model' not in locals():
    print("❌ Error: Models are not loaded. Please run the model loading cell (Cell 2) first.")
else:
    for image_name in image_files:
        display(HTML(f"<hr><h3>Debugging Image: {image_name}</h3>"))
        image_path = os.path.join(images_path, image_name)
        
        # Load image
        original_image_rgb = np.array(Image.open(image_path).convert("RGB"))
        
        # Get YOLO predictions
        yolo_results = yolo_model(original_image_rgb)
        yolo_predictions = [pred[:4].tolist() for pred in yolo_results.xyxy[0].cpu().numpy()]
        
        display(HTML(f"<b>YOLOv5 found {len(yolo_predictions)} potential objects.</b> Now classifying each one:"))
        
        if not yolo_predictions:
            display(HTML("<p>No objects to classify.</p>"))
            continue

        # Classify and display each crop
        for i, pred_box in enumerate(yolo_predictions):
            xmin, ymin, xmax, ymax = map(int, pred_box)
            crop = original_image_rgb[ymin:ymax, xmin:xmax]

            if crop.size == 0: continue

            # --- LINEA CORREGIDA ---
            # Changed cv2.COLOR_RGB_BGR to cv2.COLOR_RGB2BGR
            _, buffer = cv2.imencode('.png', cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
            crop_widget = widgets.Image(value=buffer.tobytes(), format='png', width=200)

            # Get the classification result from the helper function
            pred_class, confidence = classify_crop(crop, classifier_model, classifier_labels)

            # --- Decision Logic ---
            is_kept = POSITIVE_CLASS_NAME in pred_class and confidence >= DEBUG_CONFIDENCE_THRESHOLD
            decision_text = "KEPT" if is_kept else "DISCARDED"
            color = "green" if is_kept else "red"

            # Create the HTML widget
            result_html_string = f"""
            <div style='padding-left: 10px; font-size: 14px; border-left: 3px solid #ccc; margin-left: 5px;'>
                <p><b>Detection #{i+1}</b></p>
                <p>Classifier Result: <b>{pred_class.title()}</b></p>
                <p>Confidence: <b>{confidence:.2%}</b></p>
                <p>Decision: <b style='color:{color};'>{decision_text}</b></p>
            </div>
            """
            html_widget = widgets.HTML(value=result_html_string)
            
            # Create and display the HBox
            hbox_container = widgets.HBox([crop_widget, html_widget])
            display(hbox_container)